In [1]:
!pip install catboost 

In [2]:
#!pip install optuna
!pip install optuna catboost tensorflow imblearn

In [18]:
import os
import random
import numpy as np
import pandas as pd
import optuna
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, matthews_corrcoef, recall_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers, initializers
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

# Ensure reproducibility
def set_random_seeds(seed=1):
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_random_seeds()
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable GPU

# ✅ Hybrid Model: CatBoost + MLP + Meta-Classifier
class HybridHydroPredictor:
    def __init__(self, learning_rate=0.01, epochs=25, catboost_depth=6, top_features=10, model_weight=0.5, **kwargs): # add model_weight here
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.catboost_depth = catboost_depth
        self.top_features = top_features
        self.model_weight = model_weight # add model_weight here
        self.selected_features = None
        self.mlp_model = None
        self.catboost_model = None
        self.meta_classifier = None

    def fit(self, X_train, y_train):
        """Train CatBoost, select features, train MLP, and combine predictions using a meta-classifier."""
        smote = SMOTE(sampling_strategy="not minority", random_state=1)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        # Step 1: Train CatBoost
        self.catboost_model = CatBoostClassifier(
            depth=self.catboost_depth,
            iterations=70,
            l2_leaf_reg=9,
            learning_rate=self.learning_rate,
            random_state=1,
            verbose=0,
            task_type="CPU"
        )
        self.catboost_model.fit(X_train, y_train)

        # Step 2: Extract CatBoost Leaf Embeddings
        catboost_embeddings = self.catboost_model.calc_leaf_indexes(X_train)

        # Step 3: Train MLP on CatBoost Embeddings
        self._fit_mlp(catboost_embeddings, y_train)

        # Step 4: Train Meta-Classifier on the outputs of CatBoost and MLP
        catboost_preds = self.catboost_model.predict_proba(X_train)
        mlp_preds = self.mlp_model.predict(catboost_embeddings, verbose=0)
        combined_features = np.hstack((catboost_preds, mlp_preds))

        self.meta_classifier = LogisticRegression(random_state=1)
        self.meta_classifier.fit(combined_features, y_train)

    def _fit_mlp(self, X_train, y_train):
        """Train the MLP"""
        num_classes = len(np.unique(y_train))
        y_train = tf.keras.utils.to_categorical(y_train, num_classes)

        tf.random.set_seed(1)

        self.mlp_model = Sequential([
            tf.keras.Input(shape=(X_train.shape[1],)),
            Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)),
            BatchNormalization(),
            Dropout(0.4),
            Dense(32, activation='relu', kernel_initializer=initializers.he_normal(seed=1), kernel_regularizer=regularizers.l2(0.01)),
            BatchNormalization(),
            Dropout(0.3),
            Dense(num_classes, activation='softmax', kernel_initializer=initializers.he_normal(seed=1))
        ])

        self.mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                               loss='categorical_crossentropy',
                               metrics=['accuracy'])

        self.mlp_model.fit(X_train, y_train, epochs=self.epochs, batch_size=16, verbose=0)

    def predict(self, X):
        """Hybrid prediction using a meta-classifier"""
        catboost_embeddings = self.catboost_model.calc_leaf_indexes(X)
        catboost_preds = self.catboost_model.predict_proba(X)
        mlp_preds = self.mlp_model.predict(catboost_embeddings, verbose=0)
        combined_features = np.hstack((catboost_preds, mlp_preds))
        return self.meta_classifier.predict(combined_features)

# ✅ Data Preprocessing
def load_and_preprocess(filepath):
    df = pd.read_csv(filepath, sep=";", decimal=',')
    df = df.sort_values(by=df.columns.tolist())

    le = LabelEncoder()
    y = le.fit_transform(df["label"].values)
    X = df.drop(columns=["label"]).values

    random.seed(1)  # Lock the seed for data operations

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train = np.round(X_train, 6)
    X_test = np.round(X_test, 6)

    return X_train, X_test, y_train, y_test

# ✅ Optuna Optimization
def objective(trial, X_train, y_train, X_test, y_test):
    params = {
        'learning_rate': trial.suggest_float("learning_rate", 0.02975968404658167, 0.02975968404658167, log=True),
        'catboost_depth': trial.suggest_int("catboost_depth", 10, 10),
        'top_features': trial.suggest_int("top_features", 10, 10),
        'epochs': trial.suggest_int("epochs", 20, 20),
        'model_weight': trial.suggest_float("model_weight", 0.13725204227553417, 0.13725204227553417),
        'l2_leaf_reg': trial.suggest_int("l2_leaf_reg", 10, 10),
        'iterations': trial.suggest_int("iterations", 163, 163),
        'random_state': trial.suggest_int("random_state", 1, 1)
    }

    random.seed(1)
    set_random_seeds(1)

    model = HybridHydroPredictor(**params)

    skf = StratifiedKFold(n_splits=5, shuffle=False)
    f1_scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_fold_train, y_fold_train)
        y_pred = model.predict(X_fold_val)
        f1_scores.append(f1_score(y_fold_val, y_pred, average='weighted'))

    avg_f1 = np.mean(f1_scores)

    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    def calculate_metrics(y_true, y_pred):
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        kappa = cohen_kappa_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred, average='weighted')
        mcc = matthews_corrcoef(y_true, y_pred)
        return acc, f1, kappa, recall, mcc

    train_acc, train_f1, train_kappa, train_recall, train_mcc = calculate_metrics(y_train, y_train_pred) 
    test_acc, test_f1, test_kappa, test_recall, test_mcc = calculate_metrics(y_test, y_test_pred)
    print(f"Trial {trial.number}:, epoch {params['epochs']}, iterations {params['iterations']}, learning_rate {params['learning_rate']}, model_weight {params['model_weight']}, catboost_depth {params['catboost_depth']}, l2_leaf_reg {params['l2_leaf_reg']}" )
    print(f"Training → Acc: {train_acc:.4f}, F1: {train_f1:.4f}, Kappa: {train_kappa:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f}")    
    print(f"Testing → Acc: {test_acc:.4f}, F1: {test_f1:.4f}, Kappa: {test_kappa:.4f}, Recall: {test_recall:.4f}, MCC: {test_mcc:.4f}")

    return avg_f1

# ✅ Main Script
if __name__ == "__main__":
    filepath = "/kaggle/input/gwpmapp/Reference 3 classes without xy.csv"
    X_train, X_test, y_train, y_test = load_and_preprocess(filepath)

    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner(),
                                sampler=optuna.samplers.TPESampler(seed=1))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=50)

    best_params = study.best_trial.params
    model = HybridHydroPredictor(**best_params)

    smote = SMOTE(sampling_strategy="not minority", random_state=1)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model.fit(X_train, y_train)

    print("\n🔹 **Final Model Performance**")
    y_test_pred = model.predict(X_test)
    print(f"📌 Testing → Accuracy: {accuracy_score(y_test, y_test_pred):.4f}, F1: {f1_score(y_test, y_test_pred, average='weighted'):.4f}")


[I 2025-02-02 01:26:38,378] A new study created in memory with name: no-name-b11e860a-75a0-4077-b6a0-5d8b96dbc3b0
[I 2025-02-02 01:27:18,109] Trial 0 finished with value: 0.8452033414161878 and parameters: {'learning_rate': 0.02975968404658167, 'catboost_depth': 10, 'top_features': 10, 'epochs': 20, 'model_weight': 0.13725204227553417, 'l2_leaf_reg': 10, 'iterations': 163, 'random_state': 1}. Best is trial 0 with value: 0.8452033414161878.


Trial 0:, epoch 20, iterations 163, learning_rate 0.02975968404658167, model_weight 0.13725204227553417, catboost_depth 10, l2_leaf_reg 10
Training → Acc: 0.9073, F1: 0.9070, Kappa: 0.8610, Recall: 0.9073, MCC: 0.8631
Testing → Acc: 0.8615, F1: 0.8636, Kappa: 0.7923, Recall: 0.8615, MCC: 0.7932


[W 2025-02-02 01:27:38,203] Trial 1 failed with parameters: {'learning_rate': 0.02975968404658167, 'catboost_depth': 10, 'top_features': 10, 'epochs': 20, 'model_weight': 0.13725204227553417, 'l2_leaf_reg': 10, 'iterations': 163, 'random_state': 1} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-18-1db96564899d>", line 184, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=50)
  File "<ipython-input-18-1db96564899d>", line 151, in objective
    model.fit(X_fold_train, y_fold_train)
  File "<ipython-input-18-1db96564899d>", line 63, in fit
    self._fit_mlp(catboost_embeddings, y_train)
  File "<ipython-input-18-1db96564899d>", line 95, in _fit_mlp
    self.mlp_model.fit(X_train, y_train, epochs=self.epochs, batch_size=16, verbose=0)
  Fi

KeyboardInterrupt: 

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import optuna
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score,  matthews_corrcoef, recall_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from imblearn.over_sampling import SMOTE  # Changed to SMOTE
import matplotlib.pyplot as plt

# Ensure reproducibility
def set_random_seeds(seed=1):
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    # Disable XLA: This line ensures XLA is always disabled
    os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_random_seeds()

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Hybrid Model: MLP + CatBoost
class HybridHydroPredictor:
    def __init__(self, learning_rate=0.01, epochs=25, catboost_depth=6, model_weight=0.5, **kwargs):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.catboost_depth = catboost_depth
        self.model_weight = model_weight
        self.selected_features = None  # To store selected features
        self.mlp_model = None
        self.catboost_model = None
        
    def fit(self, X_train, y_train):
        """Train both MLP and CatBoost"""
        # Apply SMOTE before feature selection
        # Lock the random state within SMOTE:
        smote = SMOTE(sampling_strategy="not minority", random_state=1)  
        X_train, y_train = smote.fit_resample(X_train, y_train)

        # Feature selection - keep all features for now (or use alternative methods)
        self.selected_features = np.arange(X_train.shape[1])  # Keep all features
        
        # Train MLP
        self._fit_mlp(X_train, y_train)
        
        # Train CatBoost
        self.catboost_model = CatBoostClassifier(
            depth=self.catboost_depth,
            iterations=75,
            l2_leaf_reg=9,
            learning_rate=self.learning_rate,
            random_state=1,  # Lock the random state within CatBoost
            verbose=0,
            task_type="CPU"
        )
        self.catboost_model.fit(X_train, y_train)

    def _fit_mlp(self, X_train, y_train):
        """Train the MLP"""
        # Operations within layers have been moved to functions to ensure reproducibility:
        def dense_layer(units, activation, kernel_initializer, kernel_regularizer, input_shape=None):
            layer = Dense(units, activation=activation, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)
            if input_shape is not None:
                layer.build(input_shape=input_shape)  # Build the layer if input_shape is provided
            return layer

        def dropout_layer(rate, seed):
            layer = Dropout(rate, seed=seed)
            return layer
            
        num_classes = len(np.unique(y_train))
        y_train = tf.keras.utils.to_categorical(y_train, num_classes)

        # Set Seed Before MLP Model Creation
        tf.random.set_seed(1)  # Set the TensorFlow seed before creating the model

        self.mlp_model = Sequential([
            tf.keras.Input(shape=(X_train.shape[1],)),
            Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.0005, l2=0.0005)),  # Added L1/L2 here
            BatchNormalization(),
            dropout_layer(0.4, seed=1),  
            dense_layer(32, 'relu', initializers.he_normal(seed=1), regularizers.l2(0.0005)),
            BatchNormalization(),
            dropout_layer(0.3, seed=1),  
            dense_layer(num_classes, 'softmax', initializers.he_normal(seed=1), None)  # No regularizer for the output layer
        ])

        self.mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

        self.mlp_model.fit(X_train, y_train, epochs=self.epochs, batch_size=16, verbose=0)  # Always train for the given number of epochs 

    def predict(self, X):
        """Hybrid prediction"""
        X = X[:, self.selected_features]  # Use selected features for prediction
        mlp_probs = self.mlp_model.predict(X, verbose=0)
        catboost_probs = self.catboost_model.predict_proba(X)
        combined_probs = (self.model_weight * mlp_probs) + ((1 - self.model_weight) * catboost_probs)
        return np.argmax(combined_probs, axis=1)


# Data Preprocessing
def load_and_preprocess(filepath):
    df = pd.read_csv(filepath, sep=";", decimal=',')
    df = df.sort_values(by=df.columns.tolist())  # Sort for consistent order

    le = LabelEncoder()
    y = le.fit_transform(df["label"].values)
    X = df.drop(columns=["label"]).values

    # Global Seed: Set random seed at the beginning
    random.seed(1)  # Lock the seed for data operations

    # Split once, use the same split for everything
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

    # Apply StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Rounding to 6 decimal places
    X_train = np.round(X_train,6)
    X_test = np.round(X_test, 6)

    return X_train, X_test, y_train, y_test


# Optuna Optimization (Modified - Removed Caching)
def objective(trial, X_train, y_train, X_test, y_test):
    params = {
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.029703031164113368, log=True),  # Fixed interval
        'model_weight': trial.suggest_float("model_weight",  0.28892133270076946,  0.28892133270076946),  # Fixed interval
        'catboost_depth': trial.suggest_int("catboost_depth", 11,11),  # Fixed interval
    }



    # Local Seed Before Model Creation
    random.seed(1)  # Ensures consistent model initialization (CatBoost)
    set_random_seeds(1) # Ensures consistent model initialization (TensorFlow)

    model = HybridHydroPredictor(**params)

    # Perform cross-validation (same as before)
    skf = StratifiedKFold(n_splits=5, shuffle=False)  # Removed shuffle
    f1_scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

        model.fit(X_fold_train, y_fold_train)
        y_pred = model.predict(X_fold_val)
        f1_scores.append(f1_score(y_fold_val, y_pred, average='weighted'))

    avg_f1 = np.mean(f1_scores)

    # Train on full training data and evaluate on test data
    # Train on full training data and evaluate on test data
    model.fit(X_train, y_train)  # Always train on full training data
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculate metrics for training and testing
    def calculate_metrics(y_true, y_pred):
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        kappa = cohen_kappa_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred, average='weighted')
        mcc = matthews_corrcoef(y_true, y_pred)
        return acc, f1, kappa, recall, mcc

    train_acc, train_f1, train_kappa, train_recall, train_mcc = calculate_metrics(y_train, y_train_pred)
    test_acc, test_f1, test_kappa, test_recall, test_mcc = calculate_metrics(y_test, y_test_pred)

    # Print training and testing results for each trial
    print(f"Trial {trial.number}: ")
    print(f"Training → Acc: {train_acc:.4f}, F1: {train_f1:.4f}, Kappa: {train_kappa:.4f}, Recall: {train_recall:.4f}, MCC: {train_mcc:.4f} ")    
    print(f"Avg → Acc: {np.mean(train_acc):.4f}, F1: {np.mean(train_f1):.4f}, Kappa: {np.mean(train_kappa):.4f}, Recall: {np.mean(train_recall):.4f}, MCC: {np.mean(train_mcc):.4f}")
    print(f"Testing → Acc: {test_acc:.4f}, F1: {test_f1:.4f}, Kappa: {test_kappa:.4f}, Recall: {test_recall:.4f}, MCC: {test_mcc:.4f}")

    return avg_f1  # Return the average F1 score across folds

# Main Script
if __name__ == "__main__":
    filepath = "/kaggle/input/gwpmapp/Reference 3 classes without xy.csv"
    X_train, X_test, y_train, y_test = load_and_preprocess(filepath)

    study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner(),
                                sampler=optuna.samplers.TPESampler(seed=1))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=50)

    best_params = study.best_trial.params
    model = HybridHydroPredictor(**best_params)

    # Apply SMOTE to the full training set
    smote = SMOTE(sampling_strategy="not minority", random_state=1)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model.fit(X_train, y_train)

    # Evaluate the model
    print("\n🔹 **Final Model Performance**")
    y_test_pred = model.predict(X_test)
    print(
        f"📌 Testing → Accuracy: {accuracy_score(y_test, y_test_pred):.4f}, F1: {f1_score(y_test, y_test_pred, average='weighted'):.4f}")

[I 2025-02-01 18:30:30,845] A new study created in memory with name: no-name-f60fc42c-d80b-433e-abd4-5ee339950fc8
[I 2025-02-01 18:31:01,794] Trial 0 finished with value: 0.8142275499848164 and parameters: {'learning_rate': 0.029703031164113367, 'model_weight': 0.2889213327007695, 'catboost_depth': 11}. Best is trial 0 with value: 0.8142275499848164.


Trial 0: 
Training → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474 
Avg → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474
Testing → Acc: 0.8308, F1: 0.8335, Kappa: 0.7462, Recall: 0.8308, MCC: 0.7470


[I 2025-02-01 18:31:32,594] Trial 1 finished with value: 0.8142275499848164 and parameters: {'learning_rate': 0.029703031164113367, 'model_weight': 0.2889213327007695, 'catboost_depth': 11}. Best is trial 0 with value: 0.8142275499848164.


Trial 1: 
Training → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474 
Avg → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474
Testing → Acc: 0.8308, F1: 0.8335, Kappa: 0.7462, Recall: 0.8308, MCC: 0.7470


[I 2025-02-01 18:32:03,146] Trial 2 finished with value: 0.8142275499848164 and parameters: {'learning_rate': 0.029703031164113367, 'model_weight': 0.2889213327007695, 'catboost_depth': 11}. Best is trial 0 with value: 0.8142275499848164.


...

Trial 30: 
Training → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474 
Avg → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474
Testing → Acc: 0.8308, F1: 0.8335, Kappa: 0.7462, Recall: 0.8308, MCC: 0.7470


[I 2025-02-01 18:47:22,002] Trial 31 finished with value: 0.8142275499848164 and parameters: {'learning_rate': 0.029703031164113367, 'model_weight': 0.2889213327007695, 'catboost_depth': 11}. Best is trial 0 with value: 0.8142275499848164.


Trial 31: 
Training → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474 
Avg → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474
Testing → Acc: 0.8308, F1: 0.8335, Kappa: 0.7462, Recall: 0.8308, MCC: 0.7470


[I 2025-02-01 18:47:52,806] Trial 32 finished with value: 0.8142275499848164 and parameters: {'learning_rate': 0.029703031164113367, 'model_weight': 0.2889213327007695, 'catboost_depth': 11}. Best is trial 0 with value: 0.8142275499848164.


Trial 32: 
Training → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474 
Avg → Acc: 0.8958, F1: 0.8954, Kappa: 0.8436, Recall: 0.8958, MCC: 0.8474
Testing → Acc: 0.8308, F1: 0.8335, Kappa: 0.7462, Recall: 0.8308, MCC: 0.7470
